In [7]:
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
sys.path.append("C:/dev/pycmqlib/")
sys.path.append("C:/dev/pycmqlib/scripts/")

import misc
import data_handler as dh
import plotly_examples
import backtest
%matplotlib inline

In [9]:
start = datetime.date(2015, 1, 1)
end = datetime.date(2019, 12, 15)
shift_mode = 1
asset = 'j'
freq = 'd'
args = {'n': 1, 'roll_rule': '-35b', 'freq': freq, 'shift_mode': shift_mode}
args['start_date'] = start
args['end_date'] = end
df = misc.nearby(asset, **args)

df.index = pd.to_datetime(df.index)
#trace = go.Scatter(x=list(df.index),
#                   y=list(df.high))
win_s = 10
win_l = 40
df['MA1'] = dh.MA(df, n = win_s)
df['MA2'] = dh.MA(df, n = win_l)



def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.low.min(), in_view.high.max()]

win_s = 10
win_m = 40
win_l = 120
df['MA1'] = dh.MA(df, n = win_s)
df['MA2'] = dh.MA(df, n = win_m)
df['MA3'] = dh.MA(df, n = win_l)

fig = plotly_examples.plot_ohlc_with_indicators(df, ind_fields = ['MA1', 'MA2', 'MA3'])
fig.layout.on_change(zoom, 'xaxis.range')
fig

RmlndXJlV2lkZ2V0KHsKICAgICdkYXRhJzogW3snY2xvc2UnOiBhcnJheShbICA5NC4sICAxMTcuLCAgMTE5LiwgLi4uLCAxODA1LiwgMTg2MC4sIDE4NTQuXSksCiAgICAgICAgICAgICAgJ2TigKY=


In [16]:
start = datetime.date(2016, 1, 1)
end = datetime.date(2019, 12, 15)
shift_mode = 1
asset = 'j'
freq = 'm'
args = {'n': 1, 'roll_rule': '-35b', 'freq': freq, 'shift_mode': shift_mode}
args['start_date'] = start
args['end_date'] = end
mdf = misc.nearby(asset, **args)
mdf = misc.cleanup_mindata(mdf, asset)
print mdf[-5:]


                    instID exch        date  min_id    open    high     low  \
datetime                                                                      
2019-12-11 10:10:00  j2005  DCE  2019-12-11    1610  1857.0  1857.0  1856.0   
2019-12-11 10:11:00  j2005  DCE  2019-12-11    1611  1857.0  1858.0  1857.0   
2019-12-11 10:12:00  j2005  DCE  2019-12-11    1612  1857.0  1857.5  1856.0   
2019-12-11 10:13:00  j2005  DCE  2019-12-11    1613  1857.0  1857.0  1856.5   
2019-12-11 10:14:00  j2005  DCE  2019-12-11    1614  1857.0  1857.0  1856.0   

                      close  volume  openInterest contract  shift  
datetime                                                           
2019-12-11 10:10:00  1856.5     288        194266    j2005    0.0  
2019-12-11 10:11:00  1857.0     128        194346    j2005    0.0  
2019-12-11 10:12:00  1857.0     246        194448    j2005    0.0  
2019-12-11 10:13:00  1856.5      70        194480    j2005    0.0  
2019-12-11 10:14:00  1856.0     232   

In [ ]:
df = dh.conv_ohlc_freq1(mdf, '3m')

#df = mdf.copy()
#df = dh.day_split1(mdf, [300, 1500, 1630, 1930, 2100])

win_1 = 10
win_2 = 30
win_3 = 120
df['MA1'] = dh.MA(df, n = win_1)
df['MA2'] = dh.MA(df, n = win_2)
df['MA3'] = dh.MA(df, n = win_3)
#fig2 = plotly_examples.plot_ohlc_with_indicators(df, ind_fields = ['MA1', 'MA2', 'MA3'])
#fig2.layout.on_change(zoom, 'xaxis.range')
#fig2

In [ ]:
margin_rate = [0.08, 0.08]
pnl_tenors = ['1y', '2y', '3y', '4y']
cost_ratio = 1.0
offset = 1.0
start_capital = 0.0

long_signal = pd.Series(np.nan, index = df.index)
long_flag = dh.CROSSOVER2(df['MA1'],  df['MA2'], direction = 1) & (df['MA2'] >= df['MA3'])
long_signal[long_flag] = 1
cover_flag = dh.CROSSOVER2(df['MA1'],  df['MA3'], direction = -1) 
long_signal[cover_flag] = 0
long_signal = long_signal.shift(1).fillna(method='ffill').fillna(0)
short_signal = pd.Series(np.nan, index = df.index)
short_flag = dh.CROSSOVER2(df['MA1'],  df['MA2'], direction = -1) & (df['MA2'] <= df['MA3'])
short_signal[short_flag] = -1
cover_flag = dh.CROSSOVER2(df['MA1'],  df['MA3'], direction = 1)
short_signal[cover_flag] = 0
short_signal = short_signal.shift(1).fillna(method='ffill').fillna(0)
df['pos'] = (long_signal + short_signal)
df['pos'][-1] = 0.0
df['cost'] = abs(df['pos'] - df['pos'].shift(1)) * offset
df['cost'] = df['cost'].fillna(0.0)
df['traded_price'] = df.open
closed_trades = backtest.simdf_to_trades1(df, slippage = offset)
res_trade = backtest.get_trade_stats(closed_trades)
(res_pnl, ts) = backtest.get_pnl_stats([df], margin_rate, 'm', tenors = pnl_tenors, start_capital = start_capital, cost_ratio = cost_ratio)
ts[['cum_pnl']].plot()
plt.show()
print res_trade
print res_pnl